Run electronic structure calulations of P3HT repeat unit with gaussian to esp charges then build oligomer with esp charges of repeat unit

In [1]:
import os,sys

In [2]:
from streamm import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

Reading in resource json file res_local.json 


No handlers could be found for logger "streamm.resource"


In [5]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


Setup a Gaussian calculation of a single p3ht monomer built in the P3HT_build example

In [6]:
p3ht_n1 = buildingblock.Container('thiophene_hexane')
p3ht_n1.read_cply()

In [7]:
print p3ht_n1.n_particles

27


In [9]:
gaus_i = gaussian.Gaussian('p3ht_n1_gaus')

In [11]:
gaus_i.load_json()

 File not found calc_p3ht_n1_gaus.json in /Users/tkemper/Development/streamm-tools/examples 


In [12]:
gaus_i.set_strucC(p3ht_n1)

In [13]:
gaus_i.set_resource(peregrine)

In [ ]:
gaus_i.make_dir()

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'launch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
os.chdir(gaus_i.dir['launch'])

In [ ]:
gaus_i.load_str('templates','com')
gaus_i.load_str('templates','run')

In [ ]:
gaus_i.properties['commands'] = 'HF/3-21G SP Pop=MK nosym'
gaus_i.properties['charge'] = 0
gaus_i.properties['spin_mult'] = 1
gaus_i.properties['coord'] = gaus_i.strucC.write_coord()

In [ ]:
gaus_i.replacewrite_prop('com','input','com','%s.com'%(gaus_i.tag))

In [ ]:
gaus_i.properties['input_com'] = gaus_i.files['input']['com']

In [ ]:
gaus_i.properties['scratch_dir'] = gaus_i.dir['scratch']

In [ ]:
gaus_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaus_i.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

file_type = 'output'
file_key = 'fchk'
file_name = "%s.fchk"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

In [ ]:
print gaus_i.dir['home'] 

In [ ]:
os.chdir(gaus_i.dir['home'])
gaus_i.dump_json()
os.chdir(gaus_i.dir['launch'])

In [ ]:
gaus_i.meta['status'] = 'written'

In [ ]:
gaus_i.push()

In [ ]:
gaus_i.run()

In [ ]:
gaus_i.check()
print gaus_i.meta['status']

In [ ]:
gaus_i.analysis()

Set charges to ESP charges from fchk analysis

In [ ]:
gaus_i.strucC.properties['charge'] = 0.0 
for p,pt in gaus_i.strucC.particles.iteritems(): # properties["charge"] 
    pt.properties["charge"] = round(pt.properties["esp"],3)
    gaus_i.strucC.properties['charge']  += pt.properties["charge"]
    print p,pt.properties["symbol"],pt.properties["charge"]

In [ ]:
gaus_i.strucC.calc_charge()
print gaus_i.strucC.charge

Clean up charges from rounding

In [ ]:
gaus_i.strucC.particles[23].properties["charge"] += 0.001
gaus_i.strucC.particles[20].properties["charge"] += 0.001
gaus_i.strucC.particles[17].properties["charge"] += 0.001

In [ ]:
os.chdir(gaus_i.dir['home'])
gaus_i.dump_json()


In [ ]:
print gaus_i.strucC.tag

In [ ]:
p3ht_n1_esp = buildingblock.Container('thiophene_hexane_esp')

Convert to Bohr to angstroms 

In [ ]:
p3ht_n1_esp.convert_gaussian(gaus_i)

In [ ]:
p3ht_n1_esp.write_xyz()

In [ ]:
p3ht_n1_esp.write_cply()

Build oligomer with length xN

In [ ]:
xN = 10

In [ ]:
p3ht_xN = buildingblock.Container('p3ht_n%d'%(xN))
p3ht_xN += p3ht_n1_esp

In [ ]:
for n in range(1,xN):
    p3ht_xN = buildingblock.attach(p3ht_n1_esp,p3ht_xN,"T",1,"T",0)
    

In [ ]:
p3ht_xN.calc_charge()

In [ ]:
print p3ht_xN.charge

In [ ]:
p3ht_xN.tag = 'p3ht_n%d_esp'%(xN)

In [ ]:
p3ht_xN.write_xyz()

In [ ]:
p3ht_xN.write_cply()

In [ ]:
del p3ht_xN,p3ht_n1_esp,gaus_i